In [1]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import numpy as np
sns.set_style('whitegrid')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix

## Carregar os dados já tratados

In [2]:
df_leads = pd.read_csv('../bagging/aula/leads_cleaned.csv')

In [3]:
df_leads.head()

,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Search,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,A free copy of Mastering The Interview,Last Notable Activity
0,API,Olark Chat,0,0,0,0.0,0,0.0,Page Visited on Website,0,0,0,0,0,0,0,Modified
1,API,Organic Search,0,0,0,5.0,674,2.5,Email Opened,0,0,0,0,0,0,0,Email Opened
2,Landing Page Submission,Direct Traffic,0,0,1,2.0,1532,2.0,Email Opened,0,0,0,0,0,0,1,Email Opened
3,Landing Page Submission,Direct Traffic,0,0,0,1.0,305,1.0,Unreachable,0,0,0,0,0,0,0,Modified
4,Landing Page Submission,Google,0,0,1,2.0,1428,1.0,Converted to Lead,0,0,0,0,0,0,0,Modified


In [4]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9074 entries, 0 to 9073
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Lead Origin                             9074 non-null   object 
 1   Lead Source                             9074 non-null   object 
 2   Do Not Email                            9074 non-null   int64  
 3   Do Not Call                             9074 non-null   int64  
 4   Converted                               9074 non-null   int64  
 5   TotalVisits                             9074 non-null   float64
 6   Total Time Spent on Website             9074 non-null   int64  
 7   Page Views Per Visit                    9074 non-null   float64
 8   Last Activity                           9074 non-null   object 
 9   Search                                  9074 non-null   int64  
 10  Newspaper Article                       9074 non-null   int6

## Preparação dos dados

In [5]:
# Preparar os dados para o modelo
X = df_leads.drop(columns=['Converted'])
y = df_leads['Converted']

In [6]:
# Criar listas de colunas
features_numericas = X.select_dtypes(include=['number']).columns
features_categoricas = X.select_dtypes(include=['object']).columns

In [7]:
# Usar preprocessor existente
import joblib

preprocessor = joblib.load('../bagging/aula/preprocessor_dataset_leads.pkl')

In [8]:
# Dividir o dataset entre treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=51)

In [9]:
X_train = preprocessor.fit_transform(X_train).toarray()
X_test = preprocessor.transform(X_test).toarray()

In [10]:
# Mostrar os conjuntos 
print(f'Conjunto de treinamento: {X_train.shape}')
print(f'Conjunto de testes: {X_test.shape}')

Conjunto de treinamento: (7259, 68)
Conjunto de testes: (1815, 68)


## Treinamento do Modelo

In [11]:
# Criar o modelo de StackingClassifier

# Meta-Modelo
lr_model = LogisticRegression(random_state=51)

# Modelos Base
tree_model = DecisionTreeClassifier(random_state=51)
svc_model = SVC(kernel='linear')
sgd_model = SGDClassifier(penalty='elasticnet')

# Criar o Objeto do StackingClassifier

stacking_model = StackingClassifier(
    estimators=[
        ('sgd classifier', sgd_model),
        ('svc', svc_model),
        ('decision tree',tree_model)
    ],
    final_estimator=lr_model,
    passthrough=True # = False usa apenas os resultados estimadores de cada algoritmo base (Vanilla)
    # = True, usa os resultados dos estimadores de cada algoritmo base + dataset original (Blending)
)


In [12]:
# Treinar o modelo
stacking_model.fit(X_train,y_train)

,"estimators estimators: list of (str, estimator)Base estimators which will be stacked together. Each element of thelist is defined as a tuple of string (i.e. name) and an estimatorinstance. An estimator can be set to 'drop' using `set_params`.The type of estimator is generally expected to be a classifier.However, one can pass a regressor for some use case (e.g. ordinalregression).","[('sgd classifier', ...), ('svc', ...), ...]"
,"final_estimator final_estimator: estimator, default=NoneA classifier which will be used to combine the base estimators.The default classifier is a:class:`~sklearn.linear_model.LogisticRegression`.",LogisticRegre...ndom_state=51)
,"cv cv: int, cross-validation generator, iterable, or ""prefit"", default=NoneDetermines the cross-validation splitting strategy used in`cross_val_predict` to train `final_estimator`. Possible inputs forcv are:* None, to use the default 5-fold cross validation,* integer, to specify the number of folds in a (Stratified) KFold,* An object to be used as a cross-validation generator,* An iterable yielding train, test splits,* `""prefit""`, to assume the `estimators` are prefit. In this case, the estimators will not be refitted.For integer/None inputs, if the estimator is a classifier and y iseither binary or multiclass,:class:`~sklearn.model_selection.StratifiedKFold` is used.In all other cases, :class:`~sklearn.model_selection.KFold` is used.These splitters are instantiated with `shuffle=False` so the splitswill be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here.If ""prefit"" is passed, it is assumed that all `estimators` havebeen fitted already. The `final_estimator_` is trained on the `estimators`predictions on the full training set and are **not** cross validatedpredictions. Please note that if the models have been trained on the samedata to train the stacking model, there is a very high risk of overfitting... versionadded:: 1.1 The 'prefit' option was added in 1.1.. note:: A larger number of split will provide no benefits if the number of training samples is large enough. Indeed, the training time will increase. ``cv`` is not used for model evaluation but for prediction.",None
,"stack_method stack_method: {'auto', 'predict_proba', 'decision_function', 'predict'}, default='auto'Methods called for each base estimator. It can be:* if 'auto', it will try to invoke, for each estimator, `'predict_proba'`, `'decision_function'` or `'predict'` in that order.* otherwise, one of `'predict_proba'`, `'decision_function'` or `'predict'`. If the method is not implemented by the estimator, it will raise an error.",'auto'
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for `fit` of all `estimators`.`None` means 1 unless in a `joblib.parallel_backend` context. -1 meansusing all processors. See :term:`Glossary ` for more details.",None
,"passthrough passthrough: bool, default=FalseWhen False, only the predictions of estimators will be used astraining data for `final_estimator`. When True, the`final_estimator` is trained on the predictions as well as theoriginal training data.",True
,"verbose verbose: int, default=0Verbosity level.",0
,"loss loss: {'hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'}, default='hinge'The loss function to be used.- 'hinge' gives a linear SVM.- 'log_loss' gives logistic regression, a probabilistic classifier.- 'modified_huber' is another smooth loss that brings tolerance to outliers as well as probability estimates.- 'squared_hinge' is like hinge but is quadratically penalized.- 'perceptron' is the linear loss used by the perceptron algorithm.- The other losses, 'squared_error', 'huber', 'epsilon_insensitive' and 'squared_epsilon_insensitive' are designed for regression but can be useful in classification as well; see :class:`~sklearn.linear_model.SGDRegressor` for a description.M

## Avaliação do Modelo

In [13]:
y_pred = stacking_model.predict(X_test)

In [14]:
y_pred

array([1, 0, 0, ..., 0, 0, 1], shape=(1815,))

In [15]:
# Calcular métricas
accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)
f1 = f1_score(y_test,y_pred)

In [16]:
print(f'Acurácia: {accuracy}')
print(f'Precisão: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')

Acurácia: 0.7966942148760331
Precisão: 0.7463175122749591
Recall: 0.6805970149253732
F1-Score: 0.711943793911007


In [17]:
# Mostrar a matriz de confusão

conf_matrix = confusion_matrix(y_test, y_pred)

fig = px.imshow(
    conf_matrix,
    labels=dict(x = 'Predição', y='Real', color='Contagem'),
    x=['Not Converted', 'Converted'],
    y=['Not Converted', 'Converted'],
    color_continuous_scale='Viridis'
)

fig.update_traces(text=conf_matrix,texttemplate="%{z}")
fig.update_layout(coloraxis_showscale=False)

fig.show()

In [18]:
# Calcular a importância das variáveis considerando o Stacking Classifier

importances = []

for estimator in stacking_model.estimators_:

    # Modelos lineares possuem coeficiente
    if hasattr(estimator, 'coef_'):
        importances.append(np.abs(estimator.coef_[0]))
        print(f'Coeficientes do modelo {type(estimator).__name__}')
    
    # Modelos baseados em árvores
    elif hasattr(estimator, 'feature_importances_'):
        importances.append(np.abs(estimator.feature_importances_))
        print(f'Feature Importances do modelo {type(estimator).__name__}')
    
    # Caso não encontre coef e feature importances
    else:
        print(f'Não foi possível calcular a importância para {type(estimator).__name__}')

Coeficientes do modelo SGDClassifier
Coeficientes do modelo SVC
Feature Importances do modelo DecisionTreeClassifier


In [19]:
# Calcaular a média das importâncias 
importancia_media = np.mean(importances, axis=0)

In [20]:
# Obter os nomes das features
feature_names = (features_numericas.tolist() + preprocessor.named_transformers_['cat'].get_feature_names_out(features_categoricas).tolist())

In [21]:
# Criar um dataframe com nomes e importância

df_feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importancia_media})

In [22]:
df_feature_importances = df_feature_importances.sort_values(by='Importance', ascending=True)

df_feature_importances.head()

,Feature,Importance
24,Lead Source_Pay per Click Ads,0.000000
20,Lead Source_Live Chat,0.000000
51,Last Activity_Visited Booth in Tradeshow,0.000000
52,Last Notable Activity_Approached upfront,0.000000
66,Last Notable Activity_Unsubscribed,0.002266


In [23]:
# Mostrar o Ranking

fig = px.bar(df_feature_importances, x='Importance', y='Feature', orientation='h', title='Importância das Features dos algoritmos base')

fig.update_layout(height=1280,width=1000)

fig.show()

## Propriedades do Modelo

In [24]:
# Mostrar evidências do modelo
# Fazer uma predição num exemplo específico

X_sample = X_test[9].reshape(1, -1)

# Predições individuais dos estimadores
sgd_pred = stacking_model.named_estimators_['sgd classifier'].predict(X_sample)
svc_pred = stacking_model.named_estimators_['svc'].predict(X_sample)
tree_pred = stacking_model.named_estimators_['decision tree'].predict(X_sample)

# Predição final com o Stacking
stacking_pred = stacking_model.predict(X_sample)

In [25]:
# Exibir os resultados
print(f'Predição do SGD: {sgd_pred[0]}')
print(f'Predição do SVC: {svc_pred[0]}')
print(f'Predição da Árvore de Decisão: {tree_pred[0]}')
print(f'Predição Final do Stacking (Logistic Regression): {stacking_pred[0]}')

Predição do SGD: 0
Predição do SVC: 0
Predição da Árvore de Decisão: 0
Predição Final do Stacking (Logistic Regression): 0
